In [6]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

def f(x):
    return 3*x**2 - 4*x + 5

f(3.0)

ModuleNotFoundError: No module named 'numpy'

In [2]:
xs = np.arange(-5, 5, 0.25)
ys = f(xs)

plt.plot(xs, ys)


NameError: name 'np' is not defined

In [30]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0
        self._prev = set(_children)
        self._op = _op
        self.lable = label
        self._backward = lambda: None
        
    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        out = Value(self.data + other.data, _children=(self, other), _op='+')
        def _backward():
            self.grad = out.grad*1
            other.grad = out.grad*1

        self._backward = _backward
        return out
    
    def __mul__(self, other):
        out = Value(self.data * other.data, _children=(self, other), _op='*')
        def _backward():
            self.grad = out.grad*other.data
        self._backward = _backward
        return out 
    
    def tanh(self):
        x = self.data
        e = (math.exp(2*x)+1)/(math.exp(2*x)-1)
        out = Value(e, (self,), 'tanh')
        def _backward():
            self.grad = out.grad * (1-self.data**2)
        out._backward = _backward 
        return out

def lol(): # manual derivative
    h = 0.001
    
    a = Value(2.0+h, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label='f')
    L = d*f; L.label = 'L'
    L1 = L.data

    a = Value(2.0, label='a')
    b = Value(-3.0, label='b')
    c = Value(10.0, label='c')
    e = a*b; e.label = 'e'
    d = e + c; d.label = 'd'
    f = Value(-2.0, label='f')
    L = d*f; L.label = 'L'
    L2 = L.data

    print((L2-L1)/h) # 求导的定义
    
    
#a = Value(2.0)
#b = Value(-3.0)
#c = Value(10.0)
#d = a*b+c
#print(d)
#print(d._prev)
#print(d._op)

lol()

-6.000000000000227


In [31]:
# inputs x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')

# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1*w1 + x2*w2 + b
x1w1 = x1*w1; x1w1.label = 'x1*w1'
x2w2 = x2*w2; x2w2.label = 'x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
print(f"{n=}")
o = n.tanh(); o.lable = 'o'
print(f"{o=}")

n=Value(data=0.8813735870195432)
o=Value(data=1.414213562373095)


In [16]:
# o = tanh(n)
# do/dn = 1- o^2
o.grad = 1.0 # 设置最后一个节点的梯度，然后反向传播来求其他节点的梯度
# n.grad = o.grad*dtanh
n.grad = 1.0*(1-o.data**2)
x1w1x2w2.grad = n.grad*1
x1w1.grad = x1w1x2w2.grad*1
x2w2.grad = x1w1x2w2.grad*1

w1.grad = x1w1.grad*x1.data
x1.grad = x1w1.grad*w1.data # 注意这里不是 w1.grad，而是 w1.data

w2.grad = x2w2.grad*x2.data


-1.000010145687705

In [38]:
o.grad = 1.0
o._backward()
b._backward()
n._backward()
x1w1x2w2._backward()
print(f"{o.grad=}")
print(f"{n.grad=}")
print(f"{b.grad=}")
print(f"{x1w1x2w2.grad=}")
print(f"{x1w1.grad=}")
print(f"{x1.grad=}")
print(f"{w2.grad=}")

o.grad=1.0
n.grad=0.22318060010430374
b.grad=0.22318060010430374
x1w1x2w2.grad=0.22318060010430374
x1w1.grad=0
x1.grad=0
w2.grad=0
